# Zero-shot, One-shot, Few-shot Prompting 对比实验

基于吴恩达 x OpenAI 提示词工程课程的实践 notebook

## 📚 学习目标

通过精心设计的对比实验，深入理解三种提示策略：

1. **Zero-shot Prompting**（零样本提示）：不提供示例，仅凭指令
2. **One-shot Prompting**（单样本提示）：提供 1 个示例
3. **Few-shot Prompting**（少样本提示）：提供 2+ 个示例

## 🎯 实验设计理念

通过**三个不同复杂度的任务**展示：

| 任务类型 | 任务名称 | Zero-shot | One-shot | Few-shot |
|---------|---------|-----------|----------|----------|
| 🟢 简单 | 情感分析 | ✅ 够用 | ➕ 轻微提升 | ➕ 边际收益低 |
| 🟡 中等 | 信息提取 | ⚠️ 格式不稳定 | ✅ 明显改善 | ➕ 进一步优化 |
| 🔴 复杂 | 风格模仿 | ❌ 效果差 | ⚠️ 理解不足 | ✅ 接近要求 |

## 💡 核心洞察

- **渐进策略**：先试 Zero-shot，不行再加示例
- **示例质量 > 数量**：3 个精心设计的示例 > 10 个随意的示例
- **成本权衡**：每个示例都消耗 token，要在性能和成本间平衡

## 🤖 测试模型

使用 **Qwen3 8B** 模型（Q4_K_M 量化）

---

## 第一步：环境准备

安装并启动 Ollama 服务（如果已经运行过 01_principles.ipynb，可以跳过这一步）

In [1]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time
import os

print("⬇️ 正在安装 Ollama...")
install_cmd = "curl -fsSL https://ollama.com/install.sh | sh"
install_result = subprocess.run(install_cmd, shell=True, check=False)
if install_result.returncode == 0:
    print("✅ Ollama 安装完成")
else:
    print("⚠️ Ollama 安装遇到问题，继续尝试...")
    try:
        subprocess.run(["apt", "update"], check=False)
        subprocess.run(["apt", "install", "-y", "curl"], check=False)
        install_result = subprocess.run(install_cmd, shell=True, check=False)
        if install_result.returncode == 0:
            print("✅ Ollama 安装完成（重试）")
        else:
            print("⚠️ Ollama 可能已安装或安装失败，继续执行...")
    except Exception as e:
        print(f"⚠️ 备用安装方式失败: {str(e)}，继续执行...")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已在 localhost:11434 运行


In [2]:
# === 2. 下载 Qwen3 8B 模型 ===
import subprocess

print("⬇️ 正在下载 qwen3:8b...")
print("💡 提示：使用 Q4_K_M 量化，约 5.2GB，适合 T4 GPU")
pull_result = subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
if pull_result.returncode != 0:
    raise RuntimeError("❌ qwen3:8b 下载失败")

print("✅ 模型下载完成！")

⬇️ 正在下载 qwen3:8b...
💡 提示：使用 Q4_K_M 量化，约 5.2GB，适合 T4 GPU
✅ 模型下载完成！


## 第二步：导入依赖并定义核心函数

定义与模型交互的函数，以及用于对比三种 Prompting 策略的函数

In [3]:
# === 3. 导入依赖并定义核心函数 ===
import requests
import json
import time

OLLAMA_URL = "http://localhost:11434"

def get_completion(prompt, model="qwen3:8b", temperature=0.7):
    """调用 Qwen3 模型获取回复"""
    url = f"{OLLAMA_URL}/api/chat"

    options = {
        "temperature": temperature,
        "top_p": 0.9,
        "num_ctx": 8192,
        "num_predict": 4096
    }

    messages = [{"role": "user", "content": prompt}]
    payload = {"model": model, "messages": messages, "stream": True, "options": options}

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()

        result = ""
        print("🤖 模型回复: ", end='', flush=True)

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    if 'message' in data and 'content' in data['message']:
                        content = data['message']['content']
                        if content:
                            print(content, end='', flush=True)
                            result += content
                    if data.get('done', False):
                        print()
                        break
                except json.JSONDecodeError:
                    continue
        return result.strip()
    except Exception as e:
        print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]"

def compare_shots(task_name, test_input, zero_shot_prompt, one_shot_prompt, few_shot_prompt):
    """对比 Zero-shot、One-shot、Few-shot 的效果"""
    print("\n" + "="*80)
    print(f"🔬 任务：{task_name}")
    print("="*80)
    print(f"测试输入：{test_input}")
    print("="*80)

    results = {}

    # Zero-shot
    print("\n📌 Zero-shot Prompting（无示例）")
    print("─"*80)
    results['zero'] = get_completion(zero_shot_prompt)
    print(f"📊 输出长度: {len(results['zero'])} 字符")
    time.sleep(2)

    # One-shot
    print("\n📌 One-shot Prompting（1个示例）")
    print("─"*80)
    results['one'] = get_completion(one_shot_prompt)
    print(f"📊 输出长度: {len(results['one'])} 字符")
    time.sleep(2)

    # Few-shot
    print("\n📌 Few-shot Prompting（多个示例）")
    print("─"*80)
    results['few'] = get_completion(few_shot_prompt)
    print(f"📊 输出长度: {len(results['few'])} 字符")

    # 对比分析
    print("\n" + "="*80)
    print("📈 对比分析")
    print("="*80)
    print(f"Zero-shot 长度: {len(results['zero'])} 字符")
    print(f"One-shot 长度:  {len(results['one'])} 字符")
    print(f"Few-shot 长度:  {len(results['few'])} 字符")
    print("="*80 + "\n")

    return results

print("✅ 核心函数加载完成")
print("💡 可用函数: get_completion(prompt), compare_shots(...)")

✅ 核心函数加载完成
💡 可用函数: get_completion(prompt), compare_shots(...)


---

# 实验一：简单任务 - 情感分析

## 📖 任务说明

**任务**：判断产品评论的情感倾向（积极/消极/中立）

**复杂度**：🟢 简单 - 这是模型在预训练时见过大量类似数据的通用任务

## 🎯 实验假设

- **Zero-shot**：应该已经能给出正确答案（模型的基础能力）
- **One-shot**：可能在格式规范性上略有提升
- **Few-shot**：边际收益不大，因为任务本身不复杂

## 📊 测试用例

选择一条包含多种情绪色彩的评论，避免过于明显：

> "这款耳机音质还可以，但是佩戴时间长了会有点夹耳朵，价格倒是挺实惠的。"

（包含正面、负面、中性评价，考验模型的综合判断能力）

In [4]:
# 任务一：情感分析

# 测试评论（包含正负面混合的复杂情感）
test_review = "这款耳机音质还可以，但是佩戴时间长了会有点夹耳朵，价格倒是挺实惠的。"

# Zero-shot：仅给指令，不给示例
zero_shot_prompt = f"""请判断以下产品评论的整体情感倾向。

评论：{test_review}

情感倾向："""

# One-shot：提供1个示例
one_shot_prompt = f"""请判断产品评论的整体情感倾向。

[示例]
评论：电池续航太差了，用不到半天就要充电，非常不方便。
情感倾向：消极

现在判断：
评论：{test_review}
情感倾向："""

# Few-shot：提供3个示例（积极、消极、中立各一个）
few_shot_prompt = f"""请判断产品评论的整体情感倾向。

[示例1]
评论：电池续航太差了，用不到半天就要充电，非常不方便。
情感倾向：消极

[示例2]
评论：这款手机太棒了！屏幕清晰，运行流畅，强烈推荐！
情感倾向：积极

[示例3]
评论：包装还可以，发货速度一般，产品就是普通水平。
情感倾向：中立

现在判断：
评论：{test_review}
情感倾向："""

# 运行对比实验
results_task1 = compare_shots(
    task_name="情感分析（简单任务）",
    test_input=test_review,
    zero_shot_prompt=zero_shot_prompt,
    one_shot_prompt=one_shot_prompt,
    few_shot_prompt=few_shot_prompt
)


🔬 任务：情感分析（简单任务）
测试输入：这款耳机音质还可以，但是佩戴时间长了会有点夹耳朵，价格倒是挺实惠的。

📌 Zero-shot Prompting（无示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 情感倾向：中性

**分析：**  
1. **正面因素**：  
   - “音质还可以”（中性偏正面，但未强烈肯定）。  
   - “价格倒是挺实惠的”（明确正面，强调性价比）。  

2. **负面因素**：  
   - “佩戴时间长了会有点夹耳朵”（直接影响使用体验，负面较明显）。  

**结论**：  
评论中正负面因素均存在，但负面因素（佩戴不适）可能对用户体验影响较大，而正面因素（音质和价格）属于中性或次要优势。整体情感倾向偏向**中性**，既非明显积极，也非强烈消极。
📊 输出长度: 247 字符

📌 One-shot Prompting（1个示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 情感倾向：中性

**分析**：  
评论中包含正负两方面的评价：  
1. **积极因素**：音质“还可以”（中性偏积极）、价格“实惠”（明确积极）。  
2. **消极因素**：佩戴时间长会“夹耳朵”（明确负面）。  

整体来看，用户既肯定了产品的性价比，也指出了使用中的不适问题。由于负面评价（佩戴不适）直接影响用户体验，但并未使用强烈负面词汇（如“非常差”），且积极因素（价格）被明确提及，因此整体情感倾向更接近**中性**，而非完全消极。
📊 输出长度: 225 字符

📌 Few-shot Prompting（多个示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 情感倾向：中立

解析：评论中同时包含正负面因素。正面评价有"音质还可以""价格实惠"，负面评价有"佩戴时间长会夹耳朵"。正负面因素相互抵消，且没有明显的情感倾向主导，因此判定

## 📝 实验一小结

**观察要点**：

1. **Zero-shot 表现**：对于情感分析这种常见任务，Zero-shot 通常已经能给出正确的判断
2. **示例的边际效益**：One-shot 和 Few-shot 可能在输出格式上更规范，但准确度提升有限
3. **成本收益比**：在简单任务上，Zero-shot 是最高效的选择

**关键洞察**：
- ✅ **简单任务优先用 Zero-shot**，节省 token 成本
- 如果 Zero-shot 表现已经满足需求，不必盲目增加示例

---

# 实验二：中等任务 - 结构化信息提取

## 📖 任务说明

**任务**：从客户反馈中提取结构化信息（问题 + 建议），并以 XML 格式输出

**复杂度**：🟡 中等 - 需要理解语义并按特定格式输出

## 🎯 实验假设

- **Zero-shot**：能理解任务，但输出格式可能不稳定（有时是自然语言，有时是 XML）
- **One-shot**：提供一个标准示例后，格式稳定性显著提升
- **Few-shot**：进一步优化细节，但提升幅度小于 Zero → One-shot

## 📊 测试用例

选择一条包含隐含问题和建议的反馈：

> "每次打开App都要等好几秒才能加载出来，希望能优化一下启动速度，另外能不能加个深色模式？晚上用太刺眼了。"

（需要从描述中提取出"启动慢"和"缺少深色模式"两个问题，以及对应的建议）

In [5]:
# 任务二：结构化信息提取

# 测试反馈（包含多个问题和建议）
test_feedback = "每次打开App都要等好几秒才能加载出来，希望能优化一下启动速度，另外能不能加个深色模式？晚上用太刺眼了。"

# Zero-shot：仅给指令，不给示例
zero_shot_prompt = f"""从以下用户反馈中提取问题和建议，并以 XML 格式输出。

用户反馈：{test_feedback}

输出格式要求：使用 <feedback> 标签包裹，每个问题用 <issue> 标签，每个建议用 <suggestion> 标签。

XML 输出："""

# One-shot：提供1个清晰的示例
one_shot_prompt = f"""从用户反馈中提取问题和建议，并以 XML 格式输出。

[示例]
用户反馈：希望App能增加夜间模式，晚上用太刺眼了，而且字体能不能调大一点？
XML 输出：
<feedback>
  <issue>缺乏夜间模式</issue>
  <suggestion>增加夜间模式</suggestion>
  <issue>字体太小</issue>
  <suggestion>支持字体大小调节</suggestion>
</feedback>

现在处理：
用户反馈：{test_feedback}
XML 输出："""

# Few-shot：提供2个示例（覆盖不同情况）
few_shot_prompt = f"""从用户反馈中提取问题和建议，并以 XML 格式输出。

[示例1]
用户反馈：希望App能增加夜间模式，晚上用太刺眼了，而且字体能不能调大一点？
XML 输出：
<feedback>
  <issue>缺乏夜间模式</issue>
  <suggestion>增加夜间模式</suggestion>
  <issue>字体太小</issue>
  <suggestion>支持字体大小调节</suggestion>
</feedback>

[示例2]
用户反馈：支付流程太繁琐，每次都要输入好几次密码，能简化一下吗？
XML 输出：
<feedback>
  <issue>支付流程繁琐</issue>
  <suggestion>简化支付流程，减少密码输入次数</suggestion>
</feedback>

现在处理：
用户反馈：{test_feedback}
XML 输出："""

# 运行对比实验
results_task2 = compare_shots(
    task_name="结构化信息提取（中等任务）",
    test_input=test_feedback,
    zero_shot_prompt=zero_shot_prompt,
    one_shot_prompt=one_shot_prompt,
    few_shot_prompt=few_shot_prompt
)


🔬 任务：结构化信息提取（中等任务）
测试输入：每次打开App都要等好几秒才能加载出来，希望能优化一下启动速度，另外能不能加个深色模式？晚上用太刺眼了。

📌 Zero-shot Prompting（无示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: <feedback>
  <issue>每次打开App都要等好几秒才能加载出来</issue>
  <issue>晚上使用时屏幕太刺眼</issue>
  <suggestion>优化启动速度</suggestion>
  <suggestion>添加深色模式</suggestion>
</feedback>
📊 输出长度: 155 字符

📌 One-shot Prompting（1个示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: <feedback>
  <issue>启动速度慢</issue>
  <suggestion>优化启动速度</suggestion>
  <issue>缺乏深色模式</issue>
  <suggestion>增加深色模式</suggestion>
</feedback>
📊 输出长度: 137 字符

📌 Few-shot Prompting（多个示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: <feedback>
  <issue>启动速度慢</issue>
  <suggestion>优化启动速度</suggestion>
  <issue>缺乏深色模式</issue>
  <suggestion>增加深色模式</suggestion>
</feedback>
📊 输出长度: 137 字符

📈 对比分析
Zero-shot 长度: 155 字符
One-shot 长度:  137 字符
Few-shot 长度:  137 字符



## 📝 实验二小结

**观察要点**：

1. **Zero-shot 的格式问题**：虽然能理解任务，但输出格式可能不稳定（可能包含多余的解释文字，或 XML 格式不规范）
2. **One-shot 的关键作用**：一个清晰的示例能让模型准确理解期望的输出格式，稳定性大幅提升
3. **Few-shot 的优化**：多个示例能覆盖更多边缘情况，进一步提高质量

**关键洞察**：
- ✅ **中等复杂度任务建议使用 One-shot**，性价比最高
- 当输出格式有严格要求时（如 XML/JSON），示例比文字描述更有效
- One-shot → Few-shot 的提升通常小于 Zero-shot → One-shot

---

# 实验三：复杂任务 - 特定风格文案创作

## 📖 任务说明

**任务**：将产品特点转换为"古典诗意 + 现代科技感"融合的营销文案

**复杂度**：🔴 复杂 - 风格很难用语言精确描述，需要通过示例学习

## 🎯 实验假设

- **Zero-shot**：很难准确把握这种特定风格，可能写出普通文案或过度文艺的内容
- **One-shot**：开始理解风格方向，但可能只学到表面形式
- **Few-shot**：通过多个示例掌握风格的内在规律，输出更接近期望

## 📊 测试用例

产品特点：**5000mAh 大电池，支持 66W 快充**

期望风格：
- 要有古典诗意（意象、对仗、韵律感）
- 要体现现代科技（数字、性能）
- 不能太晦涩，要让人看懂
- 要有营销感染力

In [6]:
# 任务三：特定风格文案创作

# 测试产品特点
test_feature = "5000mAh 大电池，支持 66W 快充"

# Zero-shot：仅描述风格，不给示例
zero_shot_prompt = f"""请将以下手机特点转换为营销文案。

要求风格：
- 融合古典诗意（使用意象、对仗、有韵律感）
- 体现现代科技感（保留数字和性能参数）
- 既有文化底蕴又易于理解
- 具有营销感染力

手机特点：{test_feature}

营销文案："""

# One-shot：提供1个示例
one_shot_prompt = f"""将手机特点转换为特定风格的营销文案。

[示例]
手机特点：120Hz 高刷屏，10 亿色显示
营销文案：百廿帧内，光影如梭；十亿色间，万象生辉。目之所及，皆是流畅；心之所向，尽显真彩。

现在转换：
手机特点：{test_feature}
营销文案："""

# Few-shot：提供3个示例（让模型充分学习风格）
few_shot_prompt = f"""将手机特点转换为特定风格的营销文案。

[示例1]
手机特点：120Hz 高刷屏，10 亿色显示
营销文案：百廿帧内，光影如梭；十亿色间，万象生辉。目之所及，皆是流畅；心之所向，尽显真彩。

[示例2]
手机特点：1 英寸大底传感器，支持 OIS 光学防抖
营销文案：方寸之间，藏纳星河；光学稳像，凝固时光。一英寸的深邃，捕捉世界的每一丝颤动与静谧。

[示例3]
手机特点：骁龙 8 Gen3 处理器，12GB LPDDR5X 内存
营销文案：龙芯八代，算力破云；十二重天，速度无疆。科技之巅，尽在掌心；性能之极，唯我独尊。

现在转换：
手机特点：{test_feature}
营销文案："""

# 运行对比实验
results_task3 = compare_shots(
    task_name="特定风格文案创作（复杂任务）",
    test_input=test_feature,
    zero_shot_prompt=zero_shot_prompt,
    one_shot_prompt=one_shot_prompt,
    few_shot_prompt=few_shot_prompt
)


🔬 任务：特定风格文案创作（复杂任务）
测试输入：5000mAh 大电池，支持 66W 快充

📌 Zero-shot Prompting（无示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 【青锋续航·玉壶焕新】
五千毫安如江河奔涌，续航千里无忧；
六十六瓦似青锋破晓，瞬息焕新如初。

（注：以"青锋"喻快充速度，取剑锋破晓之意；"玉壶"喻电池容量，取冰清玉壶之雅。数字嵌入古典意象，对仗工整如"江河-青锋" "续航-焕新"，韵律如"涌-晓" "无忧-如初"，既显科技参数又藏诗意匠心。）
📊 输出长度: 150 字符

📌 One-shot Prompting（1个示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 营销文案：五千年续航，六十六秒焕新；持久如初，极速如电。  
（注：以“五千年”暗喻5000mAh超长续航，用“六十六秒”呼应66W快充速度，通过“如初”“如电”强化持久与极速的对比，延续示例中“数字+意象+对仗”的风格。）
📊 输出长度: 112 字符

📌 Few-shot Prompting（多个示例）
────────────────────────────────────────────────────────────────────────────────
🤖 模型回复: 营销文案：  
五千毫安，蓄势待发；六十六瓦，疾速如风。电量如海，持久续航；瞬息满电，畅享不停。  

**解析**：  
1. **数字转化**：将“5000mAh”转化为“五千毫安”，“66W”转化为“六十六瓦”，保留数字的同时增强文学性。  
2. **意象营造**：  
   - “蓄势待发”呼应大电池的持久续航，暗喻能量储备；  
   - “疾速如风”形容快充速度，突出高效便捷。  
3. **对仗与韵律**：  
   - 前两句强调容量与速度，后两句聚焦续航与体验，形成工整结构；  
   - “如海”与“如风”形成对比，强化视觉与动态感。  
4. **情感共鸣**：以“畅享不停”收尾，传

## 📝 实验三小结

**观察要点**：

1. **Zero-shot 的局限性**：对于难以用语言描述的风格，仅凭指令很难让模型理解（可能写出普通广告或过度堆砌辞藻）
2. **One-shot 的方向性**：一个示例能让模型理解大致方向，但可能只学到表面特征（如用古风词汇），未掌握内在规律
3. **Few-shot 的必要性**：多个示例让模型识别出风格的模式（如对仗、意象选择、节奏把控），输出质量接近人工水平

**关键洞察**：
- ✅ **复杂任务必须使用 Few-shot**，示例是最好的老师
- 风格模仿、创意写作、特定领域专业术语等任务，Few-shot 不可或缺
- 示例的**多样性**很重要：覆盖不同情况，让模型学到规律而非死记硬背

---

# 🎓 总结与最佳实践

## 📊 三种策略对比总结

| 策略 | 适用场景 | 优势 | 劣势 | 典型应用 |
|-----|---------|------|------|---------|
| **Zero-shot** | 🟢 简单、通用任务 | • 成本最低（token少）<br>• 响应最快<br>• 足够应对常见任务 | • 格式可能不稳定<br>• 难以传达特定风格 | • 情感分析<br>• 文本翻译<br>• 常识问答<br>• 内容总结 |
| **One-shot** | 🟡 中等复杂度任务 | • **性价比最高**<br>• 格式稳定性大幅提升<br>• 成本可控 | • 对复杂任务可能不够<br>• 示例质量要求高 | • 格式转换<br>• 结构化提取<br>• 模板填充<br>• API调用 |
| **Few-shot** | 🔴 复杂、特定任务 | • 能传达复杂模式<br>• 覆盖边缘情况<br>• 输出质量最高 | • **token消耗大**<br>• 可能超出上下文限制<br>• 示例设计成本高 | • 风格模仿<br>• 创意写作<br>• 领域专业任务<br>• 复杂推理 |

## 💡 实战决策树

```
开始新任务
    ↓
是否是常见任务（翻译、总结、分类等）？
    ├─ 是 → 使用 Zero-shot
    └─ 否 → 继续
         ↓
    是否有严格的输出格式要求（JSON/XML等）？
         ├─ 是 → 使用 One-shot（提供格式示例）
         └─ 否 → 继续
              ↓
         是否涉及风格、创意、领域专业知识？
              ├─ 是 → 使用 Few-shot（3-5个精选示例）
              └─ 否 → 先试 One-shot，不行再加示例
```

## 🎯 示例设计黄金法则

### 1. **质量 > 数量**
- ❌ 10个随意的示例 < ✅ 3个精心设计的示例
- 每个示例都应该是"教科书级别"的标准答案

### 2. **多样性原则**
- 覆盖不同情况（正例、边缘情况、甚至反例）
- 避免示例之间过于相似，防止模型过拟合

### 3. **渐进策略**
```python
# 正确的优化流程
Zero-shot（测试） → 效果不理想 → 
One-shot（加1个示例） → 还不够 → 
Few-shot（加到3-5个） → 达到要求 ✅

# 错误的做法
❌ 一上来就堆5个示例（浪费token）
❌ 效果不好就无限加示例（上下文失焦）
```

## ⚠️ 常见陷阱与解决方案

| 问题 | 原因 | 解决方案 |
|-----|------|---------|
| Few-shot效果反而比One-shot差 | 示例质量低或相互矛盾 | 检查示例一致性，保证每个都是高质量 |
| 上下文长度超限 | 示例太多太长 | 精简示例，或考虑Fine-tuning |
| 模型只会"复读"示例 | 示例过于具体，缺乏多样性 | 增加示例多样性，避免固定模板 |
| 成本太高 | 滥用Few-shot | 建立评估集，测试One-shot是否已够用 |

## 🚀 进阶技巧

### 1. **结合思维链（Chain-of-Thought）**
```python
# Few-shot + CoT，让模型边思考边模仿
[示例]
输入：...
思考过程：首先...，然后...，最后...
输出：...
```

### 2. **自适应示例选择（RAG思路）**
- 不固定示例，根据用户输入动态检索最相关的示例
- 适合处理多样化的任务

### 3. **模型能力进化**
- GPT-3.5时代：Few-shot几乎是必须的
- GPT-4/Claude时代：One-shot往往就够了
- **未来趋势**：更强的模型会进一步降低对示例的依赖

## 📈 成本优化建议

1. **建立基准测试集**：用10-20个真实案例评估不同策略的效果
2. **从简到繁**：永远先试Zero-shot，效果不好再加示例
3. **缓存示例**：把常用的Few-shot示例做成模板，避免重复设计
4. **考虑Fine-tuning**：如果需要长期大量使用Few-shot，微调模型可能更划算

---

## 🎉 实验完成

通过三个精心设计的对比实验，你已经掌握了：

✅ Zero-shot、One-shot、Few-shot的核心区别  
✅ 如何根据任务复杂度选择合适的策略  
✅ 如何设计高质量的示例  
✅ 如何在性能和成本间找到平衡  

**记住**：提示词工程的本质是**与模型的有效沟通** —— 有时一句话就够，有时需要手把手教学。关键是找到最高效的沟通方式。

💡 **下一步建议**：
- 在自己的实际项目中应用这些策略
- 建立自己的"示例库"，积累高质量示例
- 探索思维链（CoT）、思维树（ToT）等进阶技术